In [ ]:
import numpy as np
import os
import sys
import pickle
from itertools import product
import matplotlib.pyplot as plt
import deepRD.tools.trajectoryTools as trajectoryTools
import deepRD.tools.analysisTools as analysisTools
from deepRD.noiseSampler import binnedData_qi, binnedData_ri, binnedData_qiri, binnedData_qiririm

In [ ]:
parentDirectory = os.environ.get('MSMRD') + '/data/MoriZwanzig/benchmark/'
fnamebase = parentDirectory + 'simMoriZwanzig_'
localDataDirectory = './data/'

In [ ]:
# Load parameters from parameters file
parameterDictionary = analysisTools.readParameters(parentDirectory + "parameters")
# Parameters for loading continuous trajectories from files (from original simulation)
nfiles = parameterDictionary['numFiles']
dt = parameterDictionary['dt'] 
stride = parameterDictionary['stride']
totalTimeSteps = parameterDictionary['timesteps'] 
boxsize = parameterDictionary['boxsize']
boundaryType = parameterDictionary['boundaryType']
parameterDictionary

In [ ]:
# Load trajectory data from h5 files (only of distinguished particle)
trajs = []
print("Loading data ...")
for i in range(nfiles):
    traj = trajectoryTools.loadTrajectory(fnamebase, i)
    trajs.append(traj)
    sys.stdout.write("File " + str(i+1) + " of " + str(nfiles) + " done." + "\r")

In [ ]:
# Load binned data for ri+1|qi. Note one timestep from data equal parameters['dt'] * parameters['stride']
numbins = 50
lagTimesteps = 1  # Number of timesteps (from data) to look back in time 
boxsizeBinning = boxsize
qiBinnedData = binnedData_qi(boxsizeBinning, numbins, lagTimesteps)
qiBinnedData.loadData(trajs)
qiBinnedData.parameterDictionary = parameterDictionary

In [ ]:
# Dump qi binned data into pickle file
qiBinnedDataFilename = localDataDirectory + 'qiBinnedData.pickle'
pickle.dump(qiBinnedData, open(qiBinnedDataFilename, "wb" ))

In [ ]:
# Load binned data for ri+1|ri
numbins = 50
lagTimesteps = 1
riBinnedData = binnedData_ri(numbins, lagTimesteps)
riBinnedData.loadData(trajs)
riBinnedData.parameterDictionary = parameterDictionary

In [ ]:
# Dump ri binned data into pickle file
riBinnedDataFilename = localDataDirectory + 'riBinnedData.pickle'
pickle.dump(riBinnedData, open(riBinnedDataFilename, "wb" ))

## Tests

In [ ]:
qiBinnedData.nearestOccupiedNeighbour(np.array([24,24,24]))

In [ ]:
len(qiBinnedData.data[tuple([24, 24, 24])])

In [ ]:
testDist = []
for i in range(10000):
    testDist.append(qiBinnedData.sample(np.array([0.0,0.0,0.0])))

In [ ]:
labels = ['x', 'y', 'z']
for i in range(3):
    plt.plot(np.array(testDist)[:,i], 'o', alpha = 0.5, label = labels[i])
plt.legend()

In [ ]:
position = np.array([0.0,0.0,0.0])
binIndex = qiBinnedData.getBinIndex(position)
occupiedBinIndex = qiBinnedData.nearestOccupiedNeighbour(binIndex)
availableData = qiBinnedData.data[occupiedBinIndex]
print(binIndex, occupiedBinIndex)

In [ ]:
plt.plot(np.array(availableData)[:,0], 'o')

In [ ]:
for xx in np.arange(-5,5, 0.5):
    position = np.array([xx,0.0,0.0])
    binIndex = qiBinnedData.getBinIndex(position)
    occupiedBinIndex = qiBinnedData.nearestOccupiedNeighbour(binIndex)
    availableData = qiBinnedData.data[occupiedBinIndex]
    print(xx, np.mean(np.array(availableData)[:,0]))

In [ ]:
localDataDirectory = '../../data/stochasticClosure/binnedData/'
qiBinnedDataFilename = localDataDirectory + 'qiBinnedData.pickle'
#binnedDataFilename = localDataDirectory + 'riBinnedData.pickle'
qiBinnedData = pickle.load(open(qiBinnedDataFilename, "rb" ))

In [ ]:
qiriBinnedDataFilename = localDataDirectory + 'qiriBinnedData.pickle'
#binnedDataFilename = localDataDirectory + 'riBinnedData.pickle'
qiriBinnedData = pickle.load(open(qiriBinnedDataFilename, "rb" ))

In [ ]:
qiririmBinnedDataFilename = localDataDirectory + 'qiririmBinnedData.pickle'
#binnedDataFilename = localDataDirectory + 'riBinnedData.pickle'
qiririmBinnedData = pickle.load(open(qiririmBinnedDataFilename, "rb" ))

In [ ]:
qiBinnedData.nearestOccupiedNeighbour(np.array([24,24,24]))

In [ ]:
qiriBinnedData.getBinIndex([3,2,1,0.5,0.5,0.5])

In [ ]:
for array in qiririmBinnedData.occupiedTuplesArray:
    if array[3]>5:
        print(array)

In [ ]:
trajs[0][5]

In [ ]:
posIndex = 1  # Position of x coordinate in trajectory files
rIndex = 5

In [ ]:
qi = trajs[8][1000][posIndex:posIndex + 3]
ri = trajs[8][1000][rIndex:rIndex + 3]

In [ ]:
qi

In [ ]:
ri

In [ ]:
qiri = np.concatenate([qi,ri])


In [ ]:
qiri

In [ ]:
qiriBinnedData.getBinIndex(qiri)